In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os

#spark = SparkSession.builder.getOrCreate()
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonAccumulatorV2. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonAccumulatorV2([class java.lang.String, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [3]:
df = ImageSchema.readImages('C:\\Users\\John Morin\\KaylaTek\\nvsed_data\\Flight_5_with_ground_truth\\*.jpg')

In [4]:
#df = ImageSchema.readImages('C:\\Users\John Morin\\KaylaTek\\Image_Classification\\*.jpg')

In [5]:
df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)



In [6]:
df.count()

700

In [105]:
def create_bb(label_df, img):

    for column in label_df.columns:
        x = label_df[column]['box'][0]
        y = label_df[column]['box'][1]
        w = label_df[column]['box'][2] - label_df[column]['box'][0]
        h = label_df[column]['box'][3] - label_df[column]['box'][1]

        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(img,str(label_df[column]['label']),(x+w+10,y+h),0,0.3,(0,255,0))

def add_bb_column(row):
    loc = row.origin
    split_loc = loc.split('/')
    file_num = loc.split('/')[7].split('_')[1].split('.')[0]
    label_loc = os.path.join(str(split_loc[1]+'\\'), split_loc[2], split_loc[3], split_loc[4], split_loc[5], split_loc[6])
    files = os.listdir(label_loc)
    for name in files:
        if (file_num in name) and ('.json' in name):
            my_df = pd.read_json(os.path.join(label_loc, name))
            new_img = create_bb(my_df, cv2.imdecode(np.array(row.data), 1))
            return new_img

In [18]:
data = df.take(4)    # take the first four rows of the dataframe

for i in range(0,4):
    im = data[i][0]
    print("image index: {}".format(i))
    print("image type: {}, number of fields: {}".format(type(im), len(im)))
    print("image path: {}".format(im.origin))
    print("height: {}, width: {}, OpenCV type: {}".format(im.height, im.width, im.mode))
    print('Channels: {}'.format(im.nChannels))
    print("\n")

image index: 0
image type: <class 'pyspark.sql.types.Row'>, number of fields: 6
image path: file:/C:/Users/John Morin/KaylaTek/nvsed_data/Flight_5_with_ground_truth/frame_000000.jpg
height: 1024, width: 1280, OpenCV type: 0
Channels: 1


image index: 1
image type: <class 'pyspark.sql.types.Row'>, number of fields: 6
image path: file:/C:/Users/John Morin/KaylaTek/nvsed_data/Flight_5_with_ground_truth/frame_000001.jpg
height: 1024, width: 1280, OpenCV type: 0
Channels: 1


image index: 2
image type: <class 'pyspark.sql.types.Row'>, number of fields: 6
image path: file:/C:/Users/John Morin/KaylaTek/nvsed_data/Flight_5_with_ground_truth/frame_000002.jpg
height: 1024, width: 1280, OpenCV type: 0
Channels: 1


image index: 3
image type: <class 'pyspark.sql.types.Row'>, number of fields: 6
image path: file:/C:/Users/John Morin/KaylaTek/nvsed_data/Flight_5_with_ground_truth/frame_000003.jpg
height: 1024, width: 1280, OpenCV type: 0
Channels: 1




In [14]:
img = df.take(3)

for x in img:
    
    print(x.nChannels)
    print(x.mode)
    print(x.height)
    print(x.width)
#print(np.array(im.data))
#print(np.array(im.data))
#im.origin

print(np.array(cv2.imread(
    'C:\\Users\\John Morin\\KaylaTek\\nvsed_data\\Flight_5_with_ground_truth\\frame_000000.tif'
    )).ravel())

blah = add_bb_column(im)
print(type(blah))


cv2.imshow('bb_image', blah)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: nChannels

In [ ]:
img = df.take(3)

im = df.take(1)[0][0]
#im.origin

blah = add_bb_column(im)
print(type(blah))


cv2.imshow('bb_image', blah)
cv2.waitKey(0)
cv2.destroyAllWindows()
#arr = ImageSchema.toNDArray(im)
#Image.fromarray(ImageSchema.toImage(arr), 'RGB')

#cv2.imshow('image', ImageSchema.toImage(arr))


#ima = ImageSchema.toImage(ImageSchema.toNDArray(im), 'RGB')
#print(type(img[1][0].data))
#im2 = img[1][1]
#im3 = img[1][2]

#print(np.array(im.data))
#print(im.nChannels)

#arr = ImageSchema.toNDArray(im)
#arr2 = ImageSchema.toNDArray(im2)
#arr3 = ImageSchema.toNDArray(im3)


#fin = np.array(img[1][0].data)
#print(type(fin))
#print(fin.shape)



#Image.fromarray(fin, 'RGB')

#arr = ImageSchema.toNDArray(im)
#print(im.height)
#Image.fromarray(arr, "RGB")
#cv2.imshow('image', im.data)
#cv2.waitKey(0)
#cv2.destroyAllWindows()